In [ ]:
import requests
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=ImportDate%20desc&format=JSON'
response = requests.get(url)
type(response)
data = response.json()
type(data)
sitenames = []
for items in data['records']:
    sitenames.append([items['sitename'],items['datacreationdate'],items['county'],items['aqi'],items['pm2.5'],items['status'],items['latitude'],items['longitude']])
sitenames = list(sitenames)
print(sitenames)

In [13]:
sql = '''
CREATE TABLE IF NOT EXISTS records (
	id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	sitename TEXT NOT NULL,
	county TEXT,
	aqi INTEGER,
	status TEXT,
	pm25 NUMERIC,
	date TEXT,
	lat NUMERIC,
	lon NUMERIC
);
'''

import sqlite3

# Connect to SQLite database (or create it if it doesn't exist)
conn = sqlite3.connect("AQI.db")

# Create a cursor object
cursor = conn.cursor()

# Create a table
cursor.execute(sql)

# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()

In [14]:
insertSQL = '''
INSERT INTO records(sitename,county,aqi,status,pm25,date,lat,lon)
VALUES ('屏東(枋山)','屏東縣',17,'良好',6.2,'2024-10-28 09:00',22.260899,120.651472)
'''
conn = sqlite3.connect("AQI.db")

# Create a cursor object
cursor = conn.cursor()

# Create a table
cursor.execute(insertSQL)

# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()

In [19]:
import sqlite3
import requests

conn = sqlite3.connect("AQI.db")
cursor = conn.cursor()
create_table_sql = '''
CREATE TABLE IF NOT EXISTS records (
    id INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
    sitename TEXT NOT NULL,
    county TEXT,
    aqi INTEGER,
    status TEXT,
    pm25 NUMERIC,
    date TEXT,
    lat NUMERIC,
    lon NUMERIC,
    UNIQUE(sitename, date) 
);
'''
cursor.execute(create_table_sql)
url = 'https://data.moenv.gov.tw/api/v2/aqx_p_488?api_key=e8dd42e6-9b8b-43f8-991e-b3dee723a52d&limit=1000&sort=ImportDate%20desc&format=JSON'
response = requests.get(url)
data = response.json()
sitenames = []

for items in data['records']:
    sitenames.append((
        items['sitename'],
        items['county'],
        items['aqi'],
        items['status'],                      
        items['pm2.5'],
        items['datacreationdate'],            
        items['latitude'],
        items['longitude']
    ))

insertSQL = '''
INSERT INTO records (sitename, county, aqi, status, pm25, date, lat, lon)
VALUES (?, ?, ?, ?, ?, ?, ?, ?)
'''
cursor.executemany(insertSQL, sitenames)

conn.commit()
cursor.close()
conn.close()


資料庫資料清除

In [18]:
import sqlite3

# 連接到數據庫
conn = sqlite3.connect("AQI.db")
cursor = conn.cursor()

# 清除所有資料
delete_sql = 'DELETE FROM records;'
cursor.execute(delete_sql)

# 提交更改並關閉連接
conn.commit()
cursor.close()
conn.close()
